# Just Seeing GW150914 in LIGO data


## prerequisites

 * This notebook has been tested in `Python2.7` and `Python3.5`.
 * To run this notebook you will need `numpy`, `scipy`, `matplotlib`, and `h5py`.
 * To automatically download the data, this notebook uses `wget`, you can download it manually if you prefer.

In [ ]:
from __future__ import division, print_function

import h5py
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## read in the LOSC data
We will read in LIGO data in `hdf5` format available from [LOSC](https://www.losc.ligo.org/data/).
This format contains metadata that we will read.
The data read in here is 32 sec of data around GW150914 (in each of the two LIGO detectors).

In [ ]:
# download data
!wget https://www.losc.ligo.org/s/events/GW150914/H-H1_LOSC_4_V2-1126259446-32.hdf5
!wget https://www.losc.ligo.org/s/events/GW150914/L-L1_LOSC_4_V2-1126259446-32.hdf5

It's also possible to use the ASCII data files (`.txt`) provided by LOSC, reading them in with `np.loadtxt()`. In that case you would have to modify this notebook and manually enter some metadata (sample rate, duration).

This data reading code block is based on the LOSC provided [readligo.py](https://www.losc.ligo.org/s/sample_code/readligo.py) and [hdf5fits.py](https://www.losc.ligo.org/s/sample_code/hdf5fits.py) scripts.

In [ ]:
with h5py.File('H-H1_LOSC_4_V2-1126259446-32.hdf5', 'r') as dataFile:
    # read H1 strain data
    h_data = dataFile['strain']['Strain'][...]

    # get metadata
    dT = dataFile['strain']['Strain'].attrs['Xspacing']
    Tobs = dataFile['meta']['Duration'].value
    GPSstart = dataFile['meta']['GPSstart'].value

    # compute other useful metadata
    samp_rate = 1/dT
    N = len(h_data)

with h5py.File('L-L1_LOSC_4_V2-1126259446-32.hdf5', 'r') as dataFile:
    # read L1 strain data; same length, start time, etc as H1
    l_data = dataFile['strain']['Strain'][...]

In [ ]:
# make lists of sample times and positive Fourier freqs
time = np.arange(N)*dT
freq = np.fft.rfftfreq(N, d=dT)

dF = freq[1]-freq[0] # frequency spacing

## raw data

We begin by plotting the raw data to see if we can spot GW150914.

We know that GW150914 occured at about 16.4 sec in this segment.
Do to the location and orientation of the two detectors, there was a time, phase, and amplitude offset $(\Delta t, \Delta\phi, \alpha)$ between the signals observed in the two detectors.

We can overlay the two data streams to see the signal in both, if we know these offsets *a priori*:

$$\Delta t \sim 0.0070 \,\mathrm{sec}$$
$$\Delta\phi \sim 180^\circ$$
$$\alpha \sim 1.3$$

In [ ]:
fig = plt.figure(figsize=(8,4))
ax1 = fig.add_subplot(111)
ax1.plot(time, h_data, label='H data raw', color='C0')
ax1.plot(time+0.007, -1.3*l_data, label='L data raw, shifted/scaled', color='C1')
ax1.set_xlim([16.30, 16.45])
ax1.set_ylim([-0.6e-18, 0.4e-18])
ax1.set_xlabel('time (sec)')
ax1.legend(loc='lower left')

## whiten the data

The LIGO noise spectrum is **colored**, meaning there is a different noise level at each frequency.
This is in contrast to **white** noise, which has the same level for all frequencies.
In our analysis we want to down-weight frequencies with lots of noise.
We accomplish this by dividing the data by the noise spectrum.
This is called data **whitening**, which puts each measured data sample onto a scale that is directly related to the signal-to-noise ratio or SNR of that sample.
Integrating the whitened power gives the SNR-squared.

We will write a few functions to do this.
First we must compute the noise power spectral density (PSD) from the data using a running median of the Fourier transform of the data in the `get_PSD()` function.
This function also  determines "noise spikes", narrow regions of the spectrum where the noise is much greater than the median.
Next the `whiten_data()` function divides the data by the PSD.

Before computing the FFT of the data, we will taper the time domain data and set it to zero mean, which helps with numerical stability of the FFT.

In [ ]:
def get_taper(N, taplen):
    """
    calculate a taper function array of length N with the first and
    last taplen samples windowed
    :param N: Number of samples in total array
    :param taplen: number of samples to be tapered
    :return ar: a shape (N,) array containing the taper function
    """
    ar = np.ones(N, dtype=np.float64)  # populate with 1
    if taplen > 0:
        up_arg = np.arange(taplen)/taplen - 1.
        down_arg = np.arange(-(taplen-1), 1) / taplen + 1.
        ar[:taplen] = 0.5*(1. + np.cos(np.pi * up_arg))  # up taper
        ar[-taplen:] = 0.5*(1. + np.cos(np.pi * down_arg))  # down taper
    return ar


def get_PSD(fdat, df, smooth_win=16, line_model=True):
    """empirically determine noise PSD from data with median smoothing
    :param fdat: array of Fourier domain strain data
    :param df: freq spacing (Hz) of input data
    :param smooth_win: width of median smoothing window in Hz
    :param line_model: return model for "lines"
    :return PSD: noise model
    """
    MED_2_MEAN 0.69314718055994529  # convert median to mean, chi_sq 2 deg o' free

    Nfft = len(fdat)

    # raw PSD of data
    Snf = 2.*np.real(fdat*fdat.conj())
    Snf[0] = 0

    ### compute noise model
    # median smoothing
    win = int(smooth_win/df)  # size of median window
    wins = np.array( [Snf[i:i+win] for i in range(Nfft-win)] )
    meds = np.median(wins, axis=1) / MED_2_MEAN

    # pad ends to correct length
    lpad = win//2  # place first median in middle of first window
    if (Nfft-len(meds))/2 == lpad:
        rpad = lpad
    else:
        rpad = lpad+1
    PSD = np.pad(meds, (lpad,rpad),
                 'constant',
                 constant_values=(meds[0], meds[-1]))  # median noise

    if line_model:
        # find 10 sigma excursions from median
        line = Snf / PSD
        line[line<10.] = 1.
        return PSD, line
    else:
        return PSD


def whiten_data(fdat, df, flow=16., fhigh=1024.,
                psd=None, **psd_kwargs):
    """whitens data with PSD model
    :param fdat: array of Fourier domain strain data
    :param df: freq spacing (Hz) of input data
    :param flow: low freq cutoff
    :param fhigh: high freq cutoff
    :param psd: noise power spectral density model
    :param psd_kwargs: kwargs to use in get_PSD() if psd is None
    :return wdat: time domain data whitened with the computed PSD
    """

    N = 2*(len(data)-1)
    fs = np.arange(len(data))*df

    if psd is None:
        psd = get_PSD(fdat, df, psd_kwargs)

    # band pass and whiten
    scale = 2. / np.sqrt(psd)
    scale[fs<flow] = 0.
    scale[fs>fhigh] = 0.

    return fdat * scale

In [ ]:
taper_frac = 0.05  # taper 5% of data (at start and end)
taplen = int(taper_frac * N)
taper = get_taper(N, taplen)

# set zero mean and taper in place
for data in [h_data, l_data]:
    mean = data.mean()
    data -= mean
    data *= taper

# FFT data
h_four = np.fft.rfft(h_data)
l_four = np.fft.rfft(l_data)

h_PSD, h_lines = get_PSD(h_four, dF, line_model=True)
l_PSD, l_lines = get_PSD(l_four, dF, line_model=True)

h_noise = h_PSD*h_lines
l_noise = l_PSD*l_lines

h_white = whiten_data(h_four, dF, psd=h_noise)
l_white = whiten_data(l_four, dF, psd=l_noise)

We can plot our noise model.
It's easiest to interpret if we take the square root of the PSD to get an **amplitude spectral density**.
We will compare the amplitude at each frequency relative to the maximum.

In [ ]:
h_asd = np.sqrt(h_noise)
l_asd = np.sqrt(l_noise)
a0 = np.max((h_asd, l_asd))

fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
ax.loglog(freq, h_asd/a0, label='H ASD', color='C0')
ax.loglog(freq, l_asd/a0, label='L ASD', color='C1')
ax.set_xlabel('freq (Hz)')
ax.set_ylabel(r'relative amplitude specral density')
ax.set_xlim([8, 1200])
ax.set_ylim([5.0e-6, 1])
ax.legend(loc='upper right')

Now we can plot the whitened data!

In [ ]:
fig = plt.figure(figsize=(8,4))

ax1 = fig.add_subplot(111)
ax1.plot(time, h_white, label='H data, whitened', color='C0')
ax1.plot(time+0.007, -1.3*l_white, label='L data, whitened, shifted/scaled', color='C1')
ax1.set_xlim([16.30, 16.45])
ax1.set_xlabel('time (sec)')
ax1.legend(loc='lower left')

This plot panel shows the whitened Hanford data zoomed in on GW150914 with the shifted and scaled whitened Livingston data overlayed.

From the ASD plot above we saw that the noise is slightly louder in the Livingston detector, especially at low frequency.
This is one reason why the L data has lower whitened amplitude, which is a proxy for the signal-to-noise ratio (SNR).